In [ ]:
import ROOT
import glob
import math
from IPython.display import display, Markdown
from IPython.display import display, HTML
from TPCQCVis.src.drawHistograms import *
from TPCQCVis.src.drawTrending import *
from TPCQCVis.src.checkHistograms import *

display(HTML("<style>.container { width:100% !important; }</style>"))
jsroot = True

## Read root files
+ The QC output of the async Reconstruction are saved on the alien in a `QC.root` file
+ These can be downloaded with `alien_cp` to local system
+ To extract the TPC specific plots (and create Cluster plots from `ClustersData` object) use the plotQCData.C macro
+ The generated root files can be displayed like shown below (the tools displayed below cna be used to generate reports for any root file)
+ Create the needed plots and add Markdown text for explanations
+ Using `jupyter nbconvert my_dashboard.ipynb  --to html --no-input --post serve` the html report can be generated

In [ ]:
# Read the Root Files
path = "/home/berki/alice/data/asyncQC/LHC22c/apass1/"
fileList = glob.glob(path+"*_QC.root")
fileList.sort()
runList = [fileList[i][55:-8] for i in range(len(fileList))]
rootDataFile=[]
for file in fileList:
    rootDataFile.append(ROOT.TFile.Open(file,"READ"))

## Display object directly from the TFile
+ Cluster task painter generates nice canvases from the get-go, so these can simply be drawn liek shown below: 

In [ ]:
%jsroot on
objectName = "c_Q_Tot"
c = rootDataFile[0].ClusterQC.Get(objectName)
c.Draw()

display(Markdown('---\n'+'### '+objectName+ ' (shown is Run 517222)'))
display(Markdown('- Increased noise on OROC3 of C4 and A10. <br /> - Expected to be related to the powering of the field cage. <br />'+
                 '- **Both runs OK**'))

## Show mutliple instances of histograms from different runs together
+ See the drawHistograms.py for different options

In [ ]:
%jsroot on
objectName="hPt"
[hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,legendNames=runList,normalize=True,legend=True,log="logxy")
canvas.Draw()

## Create trending of variable over the runs
+ To see how a certain variable was behaving over different runs it is usually benefical to plot the trending graph
+ This can simply be generated like shown below
+ See drawTrending.py for details

In [ ]:
%jsroot on
objectName="hPt"
[trend,canvas] = drawTrending(objectName,rootDataFile,names=runList,trend="mean",error="meanError")
canvas.Draw()

In [ ]:
%jsroot on
objectName="hdEdxTotMIP_TPC;1"
[trend,canvas] = drawTrending(objectName,rootDataFile,names=runList,trend="fit(gaus,Sq,,30,70)",error="",namesFromRunList=True,axis=1)
canvas.Draw()

## Checking histograms with custom condition

Using `checkHistograms(histogram,fileList,files=-1,check="entries>0",condition=10,axis=1,debug=False)` we can do automatic quality confirmation of the histograms.

Here you can define the check condition almost anything you want using standard python and math functions. For accessing the variables of the histogram use:
+ mean
+ entries
+ stdDev
+ meanError

The quality result will be `GOOD` if the condition returns `True` (and `BAD` if the condition returns `False`)

#### Example check conditions:
+ check="entries>0"
+ check="(mean-stdDev)<50<(mean+stdDev)"
+ check="math.sqrt(abs(mean))>4"
+ check="ROOT.Round(histogram.GetMean(1)) > 47"
+ check="(histogram.Fit(\"gaus\",\"Sq\",\"\",30,60)) and 45<histogram.GetFunction(\"gaus\").GetParameter(1)<55"

if checking TH2, set `axis` to check on x or y axis. Or extract value from each axis separately using for example `histogram.GetMean(1)` 

In [ ]:
%jsroot on
objectName="hPt"
quality = checkHistograms(objectName,rootDataFile,axis=1,check="0.4<(mean+stdDev)<0.6")
[hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,legendNames=runList,check=quality,normalize=False,legend=True,log="logxy",pads=True)
canvas.Draw()

In [ ]:
%jsroot on
objectName="hdEdxTotMIP_TPC"
quality = checkHistograms(objectName,rootDataFile,axis=1,check="(histogram.Fit(\"gaus\",\"Sq\",\"\",30,70)) and 45<histogram.GetFunction(\"gaus\").GetParameter(1)<55")
[hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,legendNames=runList,check=quality,normalize=False,legend=True,log="logxy",pads=True)
canvas.Draw()